In [ ]:
# Import the required libraries and modules

%tensorflow_version 2.x

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import progressbar

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, load_model
from keras.models import Sequential
from keras.layers import Bidirectional, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, MaxPooling1D

In [ ]:
# Mount the google drive folder containing the data files

from google.colab import drive

drive.mount('/content/gdrive', force_remount = True)
os.chdir('/content/gdrive/My Drive/IntentAndNERModel')

Mounted at /content/gdrive


In [ ]:
# Read slot labels and words from CSV files

# Load the list of slot labels in to a pandas dataframe
slots_df = pd.read_csv("atis.dict.slots.csv", header = None)

# Load the list of words in to a pandas dataframe
vocab_df = pd.read_csv("atis.dict.vocab.csv", header = None)

# Initialize & compose dictionaries for slot labels (i.e. index to slot label and slot label to index)
# Use sequence numbers and slot labels as indexes of the dictionaries respectively
ind2slot_dict = {}
slot2ind_dict = {}
for i in range(len(slots_df)):
  ind2slot_dict[i] = slots_df.iloc[i, 0]
  slot2ind_dict[slots_df.iloc[i, 0]] = i

# Initialize & compose dictionaries for words (i.e. index to word and word to index)
# Use sequence numbers and slots as indexes of the dictionaries respectively
ind2word_dict = {}
word2ind_dict = {}
for i in range(len(vocab_df)):
  ind2word_dict[i] = vocab_df.iloc[i, 0]
  word2ind_dict[vocab_df.iloc[i, 0]] = i

In [ ]:
# Examine the dictionaries for sanity

print("\nIndex to slots..\n")
print(ind2slot_dict)
print("\nSlots to index..\n")
print(slot2ind_dict)

print("\nIndex to words..\n")
print(ind2word_dict)
print("\nWords to Index..\n")
print(word2ind_dict)


Index to slots..

{0: 'B-aircraft_code', 1: 'B-airline_code', 2: 'B-airline_name', 3: 'B-airport_code', 4: 'B-airport_name', 5: 'B-arrive_date.date_relative', 6: 'B-arrive_date.day_name', 7: 'B-arrive_date.day_number', 8: 'B-arrive_date.month_name', 9: 'B-arrive_date.today_relative', 10: 'B-arrive_time.end_time', 11: 'B-arrive_time.period_mod', 12: 'B-arrive_time.period_of_day', 13: 'B-arrive_time.start_time', 14: 'B-arrive_time.time', 15: 'B-arrive_time.time_relative', 16: 'B-booking_class', 17: 'B-city_name', 18: 'B-class_type', 19: 'B-compartment', 20: 'B-connect', 21: 'B-cost_relative', 22: 'B-day_name', 23: 'B-day_number', 24: 'B-days_code', 25: 'B-depart_date.date_relative', 26: 'B-depart_date.day_name', 27: 'B-depart_date.day_number', 28: 'B-depart_date.month_name', 29: 'B-depart_date.today_relative', 30: 'B-depart_date.year', 31: 'B-depart_time.end_time', 32: 'B-depart_time.period_mod', 33: 'B-depart_time.period_of_day', 34: 'B-depart_time.start_time', 35: 'B-depart_time.time'

In [ ]:
# Read the train and test data sets in to Pandas dataframes
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
# Examine the dataframes for sanity
print("\nTrain data...\n")
print(train_df.head(5))
print("\nTest data...\n")
print(test_df.head(5))


Train data...

                                               query  ... intent
0  178 479 902 851 431 444 266 240 168 210 215 23...  ...     14
1  178 916 429 228 244 444 682 851 247 654 845 60...  ...     14
2  178 916 498 827 234 849 482 739 440 435 827 15...  ...     19
3                    178 296 197 444 810 851 667 179  ...      3
4     178 730 870 415 444 682 851 678 886 12 378 179  ...      3

[5 rows x 3 columns]

Test data...

                                               query  ... intent
0  178 479 932 545 851 423 180 428 444 294 851 52...  ...     14
1  178 654 227 425 479 617 180 847 444 810 851 73...  ...      3
2  178 654 227 425 479 617 180 428 452 444 681 85...  ...     14
3  178 479 932 545 180 428 868 656 906 444 681 85...  ...     14
4  178 479 932 545 180 428 444 667 851 736 521 30...  ...     14

[5 rows x 3 columns]


In [ ]:
# Compose the Named Entity Recognition (NER) model for slot labelling

# Evaluate the No. of words in the vocabulary
vocab_cnt = len(ind2word_dict)

# Evaluate the No. of slot labels in the vocabulary
slots_cnt = len(ind2slot_dict)

# Initialize sequential API for model building
NER_Model = Sequential()
NER_Model.add(Embedding(vocab_cnt, 300))
NER_Model.add(Convolution1D(128, 5, padding = 'same', activation='relu'))
NER_Model.add(Dropout(0.20))
NER_Model.add(Bidirectional(LSTM(100,return_sequences=True)))
NER_Model.add(Bidirectional(LSTM(100,return_sequences=True)))
NER_Model.add(TimeDistributed(Dense(slots_cnt, activation='softmax')))

# Compile model & display the model summary
NER_Model.compile('rmsprop', 'categorical_crossentropy')
NER_Model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 300)         282900    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         192128    
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 200)         183200    
_________________________________________________________________
bidirectional_7 (Bidirection (None, None, 200)         240800    
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 129)         25929     
Total params: 924,957
Trainable params: 924,957
Non-trainable params: 0
________________________________________________

In [ ]:
# Train NER model

# Initialize the No. of epochs
num_epochs = 5

# For loop to iterate through the epochs
for i in range(num_epochs):
  
  
  print("\n")
  print("\nEpoch {}".format(i + 1))  

  # Include progress bar to display the training progress..
  bar = progressbar.ProgressBar(max_value=len(train_df))

  # For loop to iterate through the sentences one by one
  # Each sentence is treated as a batch
  for j in bar(range(len(train_df))):

    # Fetch the sentence
    query = train_df.loc[j, "query"]
    slots = train_df.loc[j, "slots"]

    # Convert the query and slots columns in to np.int32 arrays
    arr_query = np.array(query.split(), dtype = np.int32)
    arr_slots = np.array(slots.split(), dtype = np.int32)

    # Compose X_train by converting to a (1, n) dimension
    X_train = arr_query[np.newaxis,:]

    # Compose y_train based on one-hot encoding to a (1, n, slots_cnt) dimension
    y_train = np.eye(slots_cnt)[arr_slots][np.newaxis,:]

    # Fit the model after checking and excluding single word sentences
    if (X_train.shape[1] > 1):
      NER_Model.train_on_batch(X_train, y_train)

N/A% (0 of 4978) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--




Epoch 1


100% (4978 of 4978) |####################| Elapsed Time: 0:01:31 Time:  0:01:31
  0% (7 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:01:15




Epoch 2


100% (4978 of 4978) |####################| Elapsed Time: 0:01:21 Time:  0:01:21
  0% (7 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:01:22

...

100% (4978 of 4978) |####################| Elapsed Time: 0:01:20 Time:  0:01:20
  0% (7 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:01:18




Epoch 4


100% (4978 of 4978) |####################| Elapsed Time: 0:01:20 Time:  0:01:20
  0% (7 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:01:17




Epoch 5


100% (4978 of 4978) |####################| Elapsed Time: 0:01:20 Time:  0:01:20


In [ ]:
# Set input sentence for slot labelling
inp_sentence = input("Input Sentence: ")

print("\n")

# Split the sentence in to a list of words
word_list = inp_sentence.split()

# Initialize a list to hold the sentence's word's indices
word_list_idx = []

# Compose the list of indices for the sentence's words
for i in word_list:
  word_list_idx.append(word2ind_dict[i])

# Predict using the trained/loaded model
pred = NER_Model.predict(np.array(word_list_idx, dtype = np.int32)[np.newaxis,:])

# Print the slot labels
print("Word".ljust(15, ' '), "Index".ljust(6, ' '), "Slot Label", "\n")
for i, word in enumerate(word_list):
  print(word.ljust(15, ' '), str(word2ind_dict[word]).ljust(6, ' '), ind2slot_dict[np.argmax(pred[0][i])])

Input Sentence: what flights are there from boston to new york


Word            Index  Slot Label 

what            916    O
flights         429    O
are             228    O
there           831    O
from            444    O
boston          266    B-fromloc.city_name
to              851    O
new             619    B-toloc.city_name
york            937    I-toloc.city_name


In [ ]:
# Saving model
NER_Model.save('NER_Model.h5',overwrite = True)

In [ ]:
# Load saved model
NER_Model = load_model('NER_Model.h5')